In [1]:
%cd /ibex/user/slimhy/PADS/code
"""
Extracting features into HDF5 files for each split.
"""
import argparse
import torch

import util.misc as misc
import models.s2vs as ae_mods


def get_args_parser():
    parser = argparse.ArgumentParser("Extracting Features", add_help=False)

    # Model parameters
    parser.add_argument(
        "--batch_size",
        default=32,
        type=int,
        help="Batch size per GPU"
        " (effective batch size is batch_size * accum_iter * # gpus",
    )
    parser.add_argument(
        "--text_model_name",
        type=str,
        help="Text model name to use",
    )
    parser.add_argument(
        "--ae",
        type=str,
        metavar="MODEL",
        help="Name of autoencoder",
    )
    parser.add_argument(
        "--ae-latent-dim",
        type=int,
        default=512*8,
        help="AE latent dimension",
    )
    parser.add_argument(
        "--ae_pth",
        required=True,
        help="Autoencoder checkpoint"
    )
    parser.add_argument(
        "--point_cloud_size",
        default=2048,
        type=int,
        help="input size"
    )
    parser.add_argument(
        "--fetch_keys",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--use_embeds",
        action="store_true",
        default=False,
    )
    parser.add_argument(
        "--intensity_loss",
        action="store_true",
        default=False,
        help="Contrastive edit intensity loss using ground-truth labels.",
    )

    # Dataset parameters
    parser.add_argument(
        "--dataset",
        type=str,
        choices=["graphedits"],
        help="dataset name",
    )
    parser.add_argument(
        "--data_path",
        type=str,
        help="dataset path",
    )
    parser.add_argument(
        "--data_type",
        type=str,
        help="dataset type",
    )
    parser.add_argument(
        "--max_edge_level",
        default=None,
        type=int,
        help="maximum edge level to use",
    )
    parser.add_argument(
        "--device", default="cuda", help="device to use for training / testing"
    )
    parser.add_argument("--seed", default=0, type=int)
    parser.add_argument("--num_workers", default=60, type=int)
    parser.add_argument(
        "--pin_mem",
        action="store_true",
        help="Pin CPU memory in DataLoader for more efficient "
        "(sometimes) transfer to GPU.",
    )

    return parser


# Set dummy arg string to debug the parser
call_string = """--ae_pth ckpt/ae_m512.pth \
    --ae kl_d512_m512_l8 \
    --ae-latent-dim 4096 \
    --data_path /ibex/project/c2273/PADS/3DCoMPaT \
    --batch_size 32 \
    --num_workers 8 \
    --device cuda"""
    

# Parse the arguments
args = get_args_parser()
args = args.parse_args(call_string.split())

# --------------------
device = torch.device(args.device)

# Fix the seed for reproducibility
misc.set_all_seeds(args.seed)

torch.backends.cudnn.benchmark = True
# --------------------

# Instantiate autoencoder
ae = ae_mods.__dict__[args.ae]()
ae.eval()
print("Loading autoencoder %s" % args.ae_pth)
ae.load_state_dict(torch.load(args.ae_pth, map_location="cpu")["model"])
ae = ae.to(device)

# Compile using torch.compile
ae = torch.compile(ae, mode="max-autotune")

/ibex/user/slimhy/PADS/code
Set seed to 0
Loading autoencoder ckpt/ae_m512.pth


In [ ]:
# batch_size :: 64 pair_types :: ['rand_no_rot,rand_no_rot', 'part_drop,orig'] num_workers :: 32 use_distributed :: True num_replicas :: 2 rank :: 0 seed :: 0 shuffle :: True pin_memory :: False drop_last :: True


In [6]:
from datasets.latents import ShapeLatentDataset, ComposedPairedShapesLoader

class PairType():
    NO_ROT_PAIR = "rand_no_rot,rand_no_rot"
    PART_DROP = "part_drop,orig"

# Create your datasets
dataset_train = ShapeLatentDataset(args.data_path, split="train", shuffle_parts=True)

# batch_size :: 64
# pair_types :: ['rand_no_rot,rand_no_rot', 'part_drop,orig']
# num_workers :: 32
# use_distributed :: True
# num_replicas :: 2
# rank :: 0 seed :: 0 shuffle :: True pin_memory :: False drop_last :: True


# Create the DataLoader using the sampler
ddp_loader_0 = ComposedPairedShapesLoader(
    dataset_train,
    batch_size=64,
    pair_types_list=['rand_no_rot,rand_no_rot', 'part_drop,orig'],
    num_workers=32,
    use_distributed=True,
    num_replicas=2,
    rank=0,
    seed=0,
    shuffle=True,
    drop_last=True,
) 
ddp_loader_1 = ComposedPairedShapesLoader(
    dataset_train,
    batch_size=64,
    pair_types_list=['rand_no_rot,rand_no_rot', 'part_drop,orig'],
    num_workers=32,
    use_distributed=True,
    num_replicas=2,
    rank=1,
    seed=0,
    shuffle=True,
    drop_last=True,
) 
seq_loader = ComposedPairedShapesLoader(
    dataset_train,
    batch_size=args.batch_size,
    pair_types_list=[PairType.PART_DROP, PairType.NO_ROT_PAIR, PairType.PART_DROP],
    num_workers=0,
    shuffle=True,
    use_distributed=False
) 

/home/slimhy/conda/envs/3D2VS_flexicubes/lib/python3.10/site-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 3, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
ddp_loader_0.set_epoch(10)
ddp_loader_1.set_epoch(10)

# Use the dataloader in your training loop
for pair_types, (l_a, bb_a, bb_l_a, meta_a), (l_b, bb_b, bb_l_b, meta_b) in ddp_loader_0:
    print(meta_a[0], meta_b[0])
    break

# Use the dataloader in your training loop
for pair_types, (l_a, bb_a, bb_l_a, meta_a), (l_b, bb_b, bb_l_b, meta_b) in ddp_loader_1:
    print(meta_a[0], meta_b[0])
    break

# Use the dataloader in your training loop
for pair_types, (l_a, bb_a, bb_l_a, meta_a), (l_b, bb_b, bb_l_b, meta_b) in seq_loader:
    print(meta_a[0], meta_b[0])
    break

0c_06f_rand_no_rot_2 0c_06f_rand_no_rot_5
18_122_rand_no_rot_4 18_122_rand_no_rot_6
ayaaaa
25_0e7_part_drop_6 25_0e7_orig_0
